In [1]:
import pandas as pd
import numpy as np
import psycopg2 as pg
import pandas.io.sql as psql
from datetime import date
from sqlalchemy import create_engine

In [6]:
connection = pg.connect(host="localhost", port = 5432, database="superstore", user="postgres", password="Titbon1!")

In [7]:
sql_df = pd.read_sql_query('select * from superstore',con=connection)
print(sql_df)

      row_id        order_id  order_date   ship_date       ship_mode  \
0       1133  US-2016-114776  2016-12-06  2016-12-07     First Class   
1          1  CA-2016-152156  2016-11-08  2016-11-11    Second Class   
2          3  CA-2016-138688  2016-06-12  2016-06-16    Second Class   
3          4  US-2015-108966  2015-10-11  2015-10-18  Standard Class   
4          6  CA-2014-115812  2014-06-09  2014-06-14  Standard Class   
...      ...             ...         ...         ...             ...   
5004    9987  CA-2016-125794  2016-09-29  2016-10-03  Standard Class   
5005    9988  CA-2017-163629  2017-11-17  2017-11-21  Standard Class   
5006    9990  CA-2014-110422  2014-01-21  2014-01-23    Second Class   
5007    9991  CA-2017-121258  2017-02-26  2017-03-03  Standard Class   
5008    9994  CA-2017-119914  2017-05-04  2017-05-09    Second Class   

     customer_id     customer_name    segment        country             city  \
0       GG-14650      Greg Guthrie  Corporate  United 